In [ ]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
path = os.path.join(os.getcwd(), 'dataset')
print(os.listdir(path))
filename=[x for x in os.listdir(path) if 'Bucaramanga' in x] [0]
filename

In [ ]:
%pwd

In [ ]:
path+ '\\' +filename

In [ ]:
ab=pd.read_csv(path+ '\\' +filename,sep=',', encoding='utf-8')
ab

In [ ]:
ab.rename(columns={"ORDEN": "Código"}, inplace=True)
ab.rename(columns={"GRAVEDAD":"Clasificación"}, inplace=True)
ab.rename(columns={"Propietario de Vehículo":"Propietario"}, inplace=True)
ab.rename(columns={"COMUNA":"Detalle ubicación"}, inplace=True)

ab.info()

Duplicados

In [8]:
duplicados = ab.duplicated()
numero_duplicados = duplicados.sum()
print(f"Número de registros duplicados: {numero_duplicados}")

Número de registros duplicados: 0


Nulos

In [ ]:
nulos_por_columna = ab.isnull().sum()
print(nulos_por_columna)

In [ ]:
#!pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd

# Reemplazar "Sin información" por NaN para poder calcular los outliers 
ab['AUTOMOVIL'] = ab['AUTOMOVIL'].replace({'Sin información': np.nan})

# Convertir la columna a tipo numérico, forzando errores a NaN
ab['AUTOMOVIL'] = pd.to_numeric(ab['AUTOMOVIL'], errors='coerce')

# Calcular la media de la columna AUTOMOVIL, ignorando NaN
media_automovil = ab['AUTOMOVIL'].mean()

# Rellenar NaN con la media calculada
ab['AUTOMOVIL'].fillna(media_automovil, inplace=True)

# Calcular el IQR (Rango Intercuartílico)
Q1 = ab['AUTOMOVIL'].quantile(0.25)
Q3 = ab['AUTOMOVIL'].quantile(0.75)
IQR = Q3 - Q1

# Definir límites para identificar outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar outliers
print("Límite Inferior:", limite_inferior)
print("Límite Superior:", limite_superior)

# Identificar outliers en la columna AUTOMOVIL
outliers = ab[(ab['AUTOMOVIL'] < limite_inferior) | (ab['AUTOMOVIL'] > limite_superior)]

# Eliminar outliers del DataFrame original
tabla_cleaned = ab[~((ab['AUTOMOVIL'] < limite_inferior) | (ab['AUTOMOVIL'] > limite_superior))]

# Opcional: Mostrar el número de outliers encontrados y el DataFrame limpio
print("Número de outliers encontrados:", len(outliers))
print("DataFrame limpio:", tabla_cleaned.head())

In [ ]:
# Imprimir el número de outliers encontrados
print("Número de outliers:", len(outliers))

# Imprimir la longitud de la tabla original
print("Número de registros en la tabla original:", len(ab))

# Imprimir la longitud de la tabla sin outliers
print("Número de registros en la tabla sin outliers:", len(tabla_cleaned))

In [ ]:
# Definir el umbral
umbral = 4

# Contar cuántos registros tienen 'Sin información' en cada fila
# Suponiendo que queremos contar en todas las columnas de tipo objeto
conteo_sin_info = ab.apply(lambda row: (row == 'Sin información').sum(), axis=1)

# Filtrar registros donde el conteo de 'Sin información' es mayor que el umbral
registros_filtrados = ab[conteo_sin_info > umbral]

# Imprimir los registros filtrados
print("Registros con mayoría de 'Sin información':")
print(registros_filtrados)

In [ ]:
# Definir el umbral
umbral = 4

# Contar cuántos registros tienen 'Sin información' en cada fila
conteo_sin_info = ab.apply(lambda row: (row == 'Sin información').sum(), axis=1)

# Eliminar registros donde la mayoría de las columnas son 'Sin información'
tabla_cleaned2 = tabla_cleaned[conteo_sin_info <= umbral]

# Imprimir la longitud de la nueva tabla después de eliminar registros
print("Tabla después de eliminar registros con mayoría de 'Sin información':", len(tabla_cleaned2))

# Mostrar la nueva tabla limpia
tabla_cleaned2

In [ ]:
#!pip install fancyimpute

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt


# Si hay valores nulos, imputarlos primero

# Definir las columnas a utilizar para la detección de anomalías
columnas_a_usar = ['AUTOMOVIL', 'PEATON', 'CAMPERO', 'CAMIONETA', 
                    'MICRO', 'BUSETA', 'BUS', 'CAMION',
                    'VOLQUETA', 'MOTO', 'BICICLETA', 'OTRO']

# Filtrar el DataFrame solo con las columnas numéricas seleccionadas
data = tabla_cleaned2[columnas_a_usar]

# Inicializar el modelo Isolation Forest
model = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)

# Ajustar el modelo a los datos
model.fit(data)

# Predecir anomalías: -1 indica anomalía y 1 indica inlier
data['anomaly'] = model.predict(data)

# Mostrar los resultados
print(data.head(20))

# Visualizar los resultados (opcional)
plt.figure(figsize=(10, 6))
plt.scatter(data.index, data['AUTOMOVIL'], c=data['anomaly'], cmap='coolwarm', label='Anomalías')
plt.title('Detección de Anomalías usando Isolation Forest')
plt.xlabel('Índice')
plt.ylabel('Número de Automóviles')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from fancyimpute import IterativeImputer

# DataFrame de ejemplo con valores nulos
data = {
    'AUTOMOVIL': [1, 2, np.nan, 4, 5, np.nan],
    'PEATON': [np.nan, 1, 2, 3, np.nan, 5],
    'CAMPERO': ['a', 'b', np.nan, 'd', 'e', 'f'],
    'CAMIONETA': [np.nan, np.nan, np.nan, 4, 5, 6]
}

tabla_cleaned2 = pd.DataFrame(data)

# DataFrame original
print("DataFrame original:")
print(tabla_cleaned2)

# Calcular el porcentaje de valores nulos por columna
porcentaje_nulos = tabla_cleaned2.isnull().mean() * 100
print("\nPorcentaje de valores nulos por columna:")
print(porcentaje_nulos)

# Imputar según el porcentaje de nulos
for column in tabla_cleaned2.columns:
    if porcentaje_nulos[column] < 20:
        if tabla_cleaned2[column].dtype == 'object':
            # Imputación de moda para columnas categóricas
            imputer = SimpleImputer(strategy='most_frequent')
            # Ajustar y transformar la columna
            tabla_cleaned2[column] = imputer.fit_transform(tabla_cleaned2[[column]]).ravel()
        else:
            # Imputación de media para columnas numéricas
            imputer = SimpleImputer(strategy='mean')
            # Ajustar y transformar la columna
            tabla_cleaned2[column] = imputer.fit_transform(tabla_cleaned2[[column]]).ravel()
    else:
        # Imputación múltiple si hay un alto porcentaje de nulos (>= 20%)
        if tabla_cleaned2[column].dtype != 'object':
            iterative_imputer = IterativeImputer()
            tabla_cleaned2[column] = iterative_imputer.fit_transform(tabla_cleaned2[[column]]).ravel()

# Mostrar el DataFrame después de la imputación
print("\nDataFrame después de la imputación:")
print(tabla_cleaned2)